<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/profitability_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
code = '190650'

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime

class Stock:
  def __init__(self, code, start_page:int=1, end_page:int=26):
    self.code = code
    self.start_page = start_page
    self.end_page = end_page
    self.df = self.do_it()

  def page_to_df(self, page):
    url = "https://finance.naver.com/item/sise_day.nhn?code=" + self.code + "&page=" + str(page)
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}
    result = requests.get(url, headers = headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    tr = bs_obj.find_all("tr", {'onmouseover':'mouseOver(this)', 'onmouseout':"mouseOut(this)"})
    data_dict = {}
    for i in range(10):
      try: 
        spans = tr[i].find_all('span')
        lines = [span.get_text() for span in spans]
        lines = list(map(lambda x: x.replace(',',''), lines))

        objdate = datetime.datetime.strptime(lines[0], '%Y.%m.%d')
        data_dict[objdate] = lines[1:]
      except:
        break
      
    df = pd.DataFrame.from_dict(data_dict).transpose()
    df.columns = ['Close','_','Open',
                        'High','Low','Volume']
    df = df.drop(columns="_")

    return df

  def do_it(self):
    df = self.page_to_df(self.start_page)
    # max 9999
    # 1 page = 10 days
    for i in range(self.start_page, self.end_page): 
      try: 
        if (any(self.page_to_df(i).index != self.page_to_df(i-1).index)):
          df = df.append(self.page_to_df(i))
          print('.', end = '')
          if (i%100==0):
            print(i)
        else:
          break
      except:
        df = df.append(self.page_to_df(i))
    df = df.sort_index(axis = 0) 
    for i in ['Close', 'Open', 'High', 'Low', 'Volume']:
      df[i] = df[i].astype(int)
    return df

  def get_df(self):
    return self.df

  
  def get_code_name(self):
    url = 'https://finance.naver.com/item/main.nhn?code=' + str(self.code)
    result = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'})
    bs_obj = BeautifulSoup(result.content, "html.parser")
    name = bs_obj.find_all("div", {'class':'wrap_company'})
    return name[0].find_all('h2')[0].get_text()

In [5]:
stock = Stock(code, 1, 1)
stock.get_df()

,Close,Open,High,Low,Volume
2021-03-16,62500,63900,64900,61600,145574
2021-03-17,76400,63800,76900,62700,1220055
2021-03-18,82600,75400,85700,73600,618997
2021-03-19,89000,82500,90200,81300,429657
2021-03-22,100500,90300,100500,90300,753908
2021-03-23,99900,103000,107700,95000,690709
2021-03-24,103800,101300,103800,96000,320073
2021-03-25,133000,105000,134900,104200,1478594
2021-03-26,124300,133300,161000,120000,2938517
2021-03-29,134000,125600,134300,117500,1116440


In [76]:
url = 'https://finance.naver.com/item/main.nhn?code=' + '190650'
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}
result = requests.get(url, headers = headers)
bs_obj = BeautifulSoup(result.content, "html.parser")

ths = bs_obj.find_all("th", {'scope':'col'})
ths = [th.get_text() for th in ths]
ths

['\n\t\t\t\t\t\t\t\t\t\t\t2018.03\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t2019.03\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t2020.03\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t2021.03(E)\n', '\n\t\t\t\t\t\t\t\t\t\t\t2019.12\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t2020.03\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t2020.06\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t2020.09\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t2020.12\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t2021.03(E)\n', '\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\tIFRS연결\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\tIFRS연결\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\tIFRS별도\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\n

['매도상위',
 '거래량',
 '매수상위',
 '거개량',
 '종가',
 '전일비',
 '외국인',
 '기관',
 '최근 연간 실적',
 '최근 분기 실적',
 '\n\t\t\t\t\t\t\t\t\t\t\t2018.03\n\t\t\t\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\t\t\t\t2019.03\n\t\t\t\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\t\t\t\t2020.03\n\t\t\t\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\t\t\t\t2021.03(E)\n',
 '\n\t\t\t\t\t\t\t\t\t\t\t2019.12\n\t\t\t\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\t\t\t\t2020.03\n\t\t\t\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\t\t\t\t2020.06\n\t\t\t\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\t\t\t\t2020.09\n\t\t\t\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\t\t\t\t2020.12\n\t\t\t\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\t\t\t\t2021.03(E)\n',
 '\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\tIFRS연결\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\tIFRS연결\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t

In [ ]:
li = list(map(lambda x: x.translate(str.maketrans('','','\n\t, ')),ths[:7]))
li

In [58]:

tr = bs_obj.find_all("tbody")
tds = tr[2].find_all('td')
tds = [td.get_text() for td in tds]

li = list(map(lambda x: x.translate(str.maketrans('','','\n\t, ')),tds))
li = list(map(lambda x: float(x) if x.replace('.','').isdigit() else None, li))


[1142.0,
 1664.0,
 2183.0,
 None,
 388.0,
 860.0,
 366.0,
 414.0,
 356.0,
 None,
 84.0,
 82.0,
 94.0,
 None,
 13.0,
 34.0,
 31.0,
 57.0,
 26.0,
 None,
 56.0,
 57.0,
 65.0,
 None,
 9.0,
 25.0,
 22.0,
 41.0,
 20.0,
 None,
 7.39,
 4.95,
 4.32,
 None,
 3.45,
 3.99,
 8.41,
 13.86,
 7.24,
 None,
 4.91,
 3.41,
 2.96,
 None,
 2.21,
 2.91,
 5.95,
 9.95,
 5.57,
 None,
 None,
 11.68,
 10.9,
 None,
 None,
 None,
 None,
 None,
 14.84,
 None,
 64.4,
 59.86,
 310.66,
 None,
 643.4,
 310.66,
 1828.06,
 697.3,
 724.77,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 101.96,
 107.56,
 118.03,
 None,
 112.69,
 118.03,
 119.11,
 135.38,
 142.2,
 None,
 1183.0,
 1195.0,
 1206.0,
 None,
 153.0,
 392.0,
 341.0,
 645.0,
 310.0,
 None,
 None,
 None,
 4.5,
 None,
 None,
 None,
 None,
 4.42,
 4.44,
 None,
 10091.0,
 10378.0,
 10902.0,
 None,
 10634.0,
 10902.0,
 10955.0,
 11769.0,
 12110.0,
 None,
 None,
 None,
 0.5,
 None,
 1.05,
 0.5,
 0.62,
 0.59,
 0.62,
 None,
 None,
 None,
 300